In [1]:
import json
import math
import uuid
import random
import pprint
import pandas as pd
from datetime import datetime
from dateutil import tz
from dateutil import parser as dateparser
from faker import Faker
import fhirclient.r3.models.patient as p
import fhirclient.r3.models.humanname as hn
import fhirclient.r3.models.observation as o
import fhirclient.r3.models.quantity as q
import fhirclient.r3.models.coding as c
import fhirclient.r3.models.codeableconcept as cc
import fhirclient.r3.models.fhirdate as dt
import fhirclient.r3.models.bundle as b
import fhirclient.r3.models.fhirreference as fr
import fhirclient.r3.models.identifier as iden

pp = pprint.PrettyPrinter(indent=2)

### Make a random patient

In [2]:
# initialize fake
fake = Faker()

patient = p.Patient()

# gender
gender = random.choice(['male','female'])
patient.gender = gender

# name
name = hn.HumanName()
if gender == 'male':
    given_name = fake.first_name_male()
elif gender == 'female':
    given_name = fake.first_name_female()
family_name = fake.last_name()
name.family = family_name
# use 'Dash' as middle name so we can find it via search
name.given = [given_name, "Dash"]
name.text = given_name + " Dash " + family_name
name.use = 'official'
patient.name = [name]

# birthdate
birthdate = dt.FHIRDate()
birthdate.date = fake.date_between(start_date='-80y', end_date='-6y')
patient.birthDate = birthdate

# identifier
identifier = iden.Identifier()
identifier.system = "http://example.org"
identifier.value = uuid.uuid4().hex
typecoding = c.Coding()
typecoding.system = 'http://hl7.org/fhir/v2/0203'
typecoding.code = 'MR'
typecoding.display = 'Medical record number'
itype = cc.CodeableConcept()
itype.coding = [typecoding]
identifier.type = itype
patient.identifier = [identifier]

print(json.dumps(patient.as_json(), indent=2))

{
  "birthDate": "1987-03-20",
  "gender": "male",
  "identifier": [
    {
      "system": "http://example.org",
      "type": {
        "coding": [
          {
            "code": "MR",
            "display": "Medical record number",
            "system": "http://hl7.org/fhir/v2/0203"
          }
        ]
      },
      "value": "a834ec6b0db74de2a2223be5fbce669b"
    }
  ],
  "name": [
    {
      "family": "Torres",
      "given": [
        "Russell",
        "Dash"
      ],
      "text": "Russell Dash Torres",
      "use": "official"
    }
  ],
  "resourceType": "Patient"
}


### Create a patient uuid that other resources can reference later in a bundle

In [3]:
patient.uuid = uuid.uuid4().urn
print(patient.uuid)
pp.pprint(patient.__dict__)
print(type(patient))

urn:uuid:6183aea0-f1b5-4f88-bc7e-b8f5b3feb4a8
{ '_owner': None,
  '_resolved': None,
  '_server': None,
  'active': None,
  'address': None,
  'animal': None,
  'birthDate': <fhirclient.r3.models.fhirdate.FHIRDate object at 0x1174f4190>,
  'communication': None,
  'contact': None,
  'contained': None,
  'deceasedBoolean': None,
  'deceasedDateTime': None,
  'extension': None,
  'gender': 'male',
  'generalPractitioner': None,
  'id': None,
  'identifier': [ <fhirclient.r3.models.identifier.Identifier object at 0x1174f42b0>],
  'implicitRules': None,
  'language': None,
  'link': None,
  'managingOrganization': None,
  'maritalStatus': None,
  'meta': None,
  'modifierExtension': None,
  'multipleBirthBoolean': None,
  'multipleBirthInteger': None,
  'name': [<fhirclient.r3.models.humanname.HumanName object at 0x1174f4220>],
  'photo': None,
  'telecom': None,
  'text': None,
  'uuid': 'urn:uuid:6183aea0-f1b5-4f88-bc7e-b8f5b3feb4a8'}
<class 'fhirclient.r3.models.patient.Patient'>


### Create a transaction bundle and add the patient to it

In [4]:
bundle = b.Bundle()
bundle.type = 'transaction'
bundleEntry = b.BundleEntry()
bundleEntry.fullUrl = patient.uuid
bundleEntry.resource = patient
bundleEntryRequest = b.BundleEntryRequest()
bundleEntryRequest.method = "POST"
bundleEntryRequest.url = "Patient"
bundleEntry.request = bundleEntryRequest
bundle.entry = [bundleEntry]
print(json.dumps(bundle.as_json(), indent=2))

{
  "entry": [
    {
      "fullUrl": "urn:uuid:6183aea0-f1b5-4f88-bc7e-b8f5b3feb4a8",
      "request": {
        "method": "POST",
        "url": "Patient"
      },
      "resource": {
        "birthDate": "1987-03-20",
        "gender": "male",
        "identifier": [
          {
            "system": "http://example.org",
            "type": {
              "coding": [
                {
                  "code": "MR",
                  "display": "Medical record number",
                  "system": "http://hl7.org/fhir/v2/0203"
                }
              ]
            },
            "value": "a834ec6b0db74de2a2223be5fbce669b"
          }
        ],
        "name": [
          {
            "family": "Torres",
            "given": [
              "Russell",
              "Dash"
            ],
            "text": "Russell Dash Torres",
            "use": "official"
          }
        ],
        "resourceType": "Patient"
      }
    }
  ],
  "type": "transaction",
  "resourceType

### Create a subject reference object that can reused in other resources in bundle

In [5]:
subject = fr.FHIRReference()
subject.reference = patient.uuid
print(json.dumps(subject.as_json(), indent=2))

{
  "reference": "urn:uuid:6183aea0-f1b5-4f88-bc7e-b8f5b3feb4a8"
}


### Read CSV file containing LOINC data

In [6]:
file = "loinc_2.csv"
df = pd.read_csv(file)
df.head(5)

,LOINC Code,LOINC Concept Name,Value[x],value,q.unit,q.system,q.code,cc.code,cc.system,cc.display,q.low,q.high,Unnamed: 12
0,26508-2,Band form neutrophils/100 leukocytes in Blood,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.0,6.0,https://healthmatters.io/understand-blood-test...
1,35332-6,Band form neutrophils/100 leukocytes in Blood ...,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.0,6.0,https://healthmatters.io/understand-blood-test...
2,764-1,Band form neutrophils/100 leukocytes in Blood ...,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.0,6.0,https://healthmatters.io/understand-blood-test...
3,30180-4,Basophils/100 leukocytes in Blood,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.5,1.0,https://www.ucsfhealth.org/medical-tests/blood...
4,706-2,Basophils/100 leukocytes in Blood by Automated...,Quantity,NaN,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN,0.5,1.0,https://www.ucsfhealth.org/medical-tests/blood...


### Create observation resource using data from CSV file with LOINC codes

In [7]:
for index, row in df.iterrows():
    if row['Value[x]'] == 'Quantity':
        # for each LOINC code
        for i in range(random.randint(1, 10)):
            observation = o.Observation()
            
            # status (required)
            observation.status = "final"
        
            # code (required)
            code = cc.CodeableConcept()
            codecode = c.Coding()
            codecode.system = 'http://loinc.org'
            codecode.code = row['LOINC Code']
            codecode.display = row['LOINC Concept Name']
            code.text = row['LOINC Concept Name']
            code.coding = [codecode]
            observation.code = code

            # category
            category = cc.CodeableConcept()
            categorycoding = c.Coding()
            categorycoding.system = "http://loinc.org"
            categorycoding.code = "laboratory"
            categorycoding.display = "Laboratory"
            category.coding = [categorycoding]
            observation.category = [category]

            # subject
            observation.subject = subject
                    
            # date, sometime between birthdate and now
            effectiveDateTime = dt.FHIRDate()
            effectiveDateTime.date = fake.date_time_between(start_date=birthdate.date, end_date='now', tzinfo=tz.UTC)
            observation.effectiveDateTime = effectiveDateTime
            
            # value
            low = row['q.low']
            high = row['q.high']
            value = math.ceil(random.uniform(low, high)*10)/10
            valuequantity = q.Quantity()
            valuequantity.value = value
            valuequantity.unit = row['q.unit']
            valuequantity.system = row['q.system']
            valuequantity.code = row['q.code']
            observation.valueQuantity = valuequantity
            
            # put observation into bundle entry
            bundleEntry = b.BundleEntry()
            bundleEntryRequest = b.BundleEntryRequest()
            bundleEntryRequest.method = "POST"
            bundleEntryRequest.url = "Observation"
            bundleEntry.request = bundleEntryRequest
            bundleEntry.resource = observation
            bundleEntry.fullUrl = uuid.uuid4().urn
            bundle.entry.append(bundleEntry)      

print(json.dumps(bundle.as_json(), indent=2))

{
  "entry": [
    {
      "fullUrl": "urn:uuid:6183aea0-f1b5-4f88-bc7e-b8f5b3feb4a8",
      "request": {
        "method": "POST",
        "url": "Patient"
      },
      "resource": {
        "birthDate": "1987-03-20",
        "gender": "male",
        "identifier": [
          {
            "system": "http://example.org",
            "type": {
              "coding": [
                {
                  "code": "MR",
                  "display": "Medical record number",
                  "system": "http://hl7.org/fhir/v2/0203"
                }
              ]
            },
            "value": "a834ec6b0db74de2a2223be5fbce669b"
          }
        ],
        "name": [
          {
            "family": "Torres",
            "given": [
              "Russell",
              "Dash"
            ],
            "text": "Russell Dash Torres",
            "use": "official"
          }
        ],
        "resourceType": "Patient"
      }
    },
    {
      "fullUrl": "urn:uuid:b1ba45e2-85

### sort the observations on effectiveDateTime
I did this to see if it would affect the order of search result in the smart app. It didn't, so it isn't necessary, but it was a good learning exercise.

In [8]:
# # bundle is not subscriptable, so need to copy it into a dict before we can sort it
# bundlejson = json.dumps(bundle.as_json())
# bundledict = json.loads(bundlejson)
# # put all the observations into its own list
# bundleobs = []
# for i in bundledict['entry']:
#     if i['resource']['resourceType'] == "Observation":
#         bundleobs.append(i)
# # print(json.dumps(bundleobs[0], indent=2))
# print("before = " + str(dateparser.isoparse(bundleobs[0]['resource']['effectiveDateTime'])))
# newObs = sorted(bundleobs, key = lambda x: dateparser.isoparse(x['resource']['effectiveDateTime']), reverse=True)
# print("sorted = " + str(dateparser.isoparse(newObs[0]['resource']['effectiveDateTime'])))
# # print(json.dumps(tem[0], indent=2))

### replace bundle observations with sorted

In [9]:
# newBundleEntry = b.BundleEntry()
# newBundleEntry.fullUrl = patient.uuid
# newBundleEntry.resource = patient
# newBundleEntryRequest = b.BundleEntryRequest()
# newBundleEntryRequest.method = "POST"
# newBundleEntryRequest.url = "Patient"
# newBundleEntry.request = newBundleEntryRequest
# # print(json.dumps(newBundleEntry.as_json(), indent=2))
# # print(" ")

# bundle.entry = [newBundleEntry]
# bundle.entry.extend(newObs)

# # bundle.entry = newBundleEntry
# print(json.dumps(bundle.as_json(), indent=2))

### Write to file

In [10]:
with open('loinc_bundle9.json', 'w') as outfile:
    json.dump(bundle.as_json(), outfile, indent=2)

***
## Scratchpad below

In [11]:
data = [
    {"name": "Shawn", "date": "2019-12-07"}, 
    {"name": "John", "date": "2019-12-05"}, 
    {"name": "Dave", "date": "2019-12-01"}
]
print(data)
tem = sorted(
    data,
    key = lambda x: datetime.strptime(x["date"], '%Y-%m-%d')
)
print(tem)

[{'name': 'Shawn', 'date': '2019-12-07'}, {'name': 'John', 'date': '2019-12-05'}, {'name': 'Dave', 'date': '2019-12-01'}]
[{'name': 'Dave', 'date': '2019-12-01'}, {'name': 'John', 'date': '2019-12-05'}, {'name': 'Shawn', 'date': '2019-12-07'}]


In [12]:
uuid.uuid4().urn

'urn:uuid:6ae71bb3-e59a-42d1-bf59-fc72a87631d2'

In [13]:
uuid.uuid4().hex

'3570c64d8d164f2c9b335958decc8122'

In [14]:
fake = Faker()

In [15]:
fake.name()

'Desiree Weaver'

In [16]:
fake.address()

'39354 Cheyenne Ferry Apt. 562\nWest Kristopher, IL 91408'

In [17]:
for i in range(10):
    print(fake.name())

Thomas Prince
Donald Russell
Jeffrey Washington
Matthew Williams
Laura Atkinson
Allison Berry
Gary Hughes
Daniel Brewer
Jeffrey Lee
Amber Juarez


In [18]:
fake.date_time_between(start_date='-5y', end_date='now').isoformat()

'2020-09-22T23:59:49'

In [19]:
fakedate = fake.date_time_between(start_date='-10y', end_date='now', tzinfo=tz.UTC).isoformat()
print(fakedate)
edt = dt.FHIRDate()
edt = datetime.fromisoformat(fakedate)
print(edt)

2017-07-19T03:15:08+00:00
2017-07-19 03:15:08+00:00


In [20]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2021-07-09T21:42:35.282606
